In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping 

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPooling1D, SimpleRNN, LSTM, GRU, Masking

In [3]:
from string import punctuation
import re

In [4]:
path_to_file = 'evgenyi_onegin.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [6]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [7]:
# sw = set(get_stop_words("ru"))
exclude = set(punctuation)

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()

#     txt = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•`'·&()]|[+=]|[[]|[]]|[/]|", '', txt)
#     txt = re.sub("[+=]|[[]|[]]|[/]|", '', txt)
#     txt = re.sub("\\'", "'", txt)
    txt = re.sub("[0-9]|[a-zA-z]", '', txt)
    txt = re.sub(r"i+", '', txt)
    txt = re.sub(r"i+|v+|x+", '', txt)
    txt = re.sub(r"\s+", ' ', txt)
    txt = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', txt)
    txt = re.sub(r'[\xad]', '', txt.strip())

#     txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return "".join(txt)

In [8]:
text = preprocess_text(text)

In [9]:
text

'александр сергеевич пушкин евгений онегин роман в стихах не мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных небрежный плод моих забав бессонниц легких вдохновений незрелых и увядших лет ума холодных наблюдений и сердца горестных замет глава первая и жить торопится и чувствовать спешит кн вяземский мой дядя самых честных правил когда не в шутку занемог он уважать себя заставил и лучше выдумать не мог его пример другим наука но боже мой какая скука с больным сидеть и день и ночь не отходя ни шагу прочь какое низкое коварство полуживого забавлять ему подушки поправлять печально подносить лекарство вздыхать и думать про себя когда же черт возьмет тебя так думал молодой повеса летя в пыли на почтовых всевышней волею зевеса наследник вс

In [10]:
# txt_ = text.split()
# txt_.append(' ')

In [11]:
vocab = sorted(set(text.split()))
len(vocab), vocab

(8366,
 ['а',
  'авось',
  'авроры',
  'автомедоны',
  'автор',
  'ага',
  'агафон',
  'ада',
  'адама',
  'адриатические',
  'адских',
  'ае',
  'азбуке',
  'азбучным',
  'академиком',
  'академический',
  'акаций',
  'аккорды',
  'аккуратный',
  'актрис',
  'акулькой',
  'алей',
  'александр',
  'алина',
  'алину',
  'алкало',
  'аллее',
  'аллей',
  'аллею',
  'алтаре',
  'алтарем',
  'альбана',
  'альбиона',
  'альбом',
  'альбома',
  'альбомах',
  'альбоме',
  'альбомы',
  'альманаха',
  'альманахи',
  'амуры',
  'ананасом',
  'анахоретом',
  'ангел',
  'английскому',
  'андрюшка',
  'анекдоты',
  'анисье',
  'анисья',
  'аонид',
  'аполлона',
  'аппетит',
  'аптеки',
  'апулея',
  'аристократов',
  'армейский',
  'армид',
  'архивны',
  'аршин',
  'атаман',
  'африки',
  'ах',
  'б',
  'баб',
  'бабушкам',
  'бабушки',
  'бабы',
  'багряною',
  'багряный',
  'байрон',
  'байрона',
  'бал',
  'бала',
  'бале',
  'балет',
  'балеты',
  'балконе',
  'балконы',
  'балов',
  'балтичес

In [12]:
# text = text.split()

In [13]:
# Creating a mapping from unique word to indices
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in text.split()])

In [14]:
word2idx

{'а': 0,
 'авось': 1,
 'авроры': 2,
 'автомедоны': 3,
 'автор': 4,
 'ага': 5,
 'агафон': 6,
 'ада': 7,
 'адама': 8,
 'адриатические': 9,
 'адских': 10,
 'ае': 11,
 'азбуке': 12,
 'азбучным': 13,
 'академиком': 14,
 'академический': 15,
 'акаций': 16,
 'аккорды': 17,
 'аккуратный': 18,
 'актрис': 19,
 'акулькой': 20,
 'алей': 21,
 'александр': 22,
 'алина': 23,
 'алину': 24,
 'алкало': 25,
 'аллее': 26,
 'аллей': 27,
 'аллею': 28,
 'алтаре': 29,
 'алтарем': 30,
 'альбана': 31,
 'альбиона': 32,
 'альбом': 33,
 'альбома': 34,
 'альбомах': 35,
 'альбоме': 36,
 'альбомы': 37,
 'альманаха': 38,
 'альманахи': 39,
 'амуры': 40,
 'ананасом': 41,
 'анахоретом': 42,
 'ангел': 43,
 'английскому': 44,
 'андрюшка': 45,
 'анекдоты': 46,
 'анисье': 47,
 'анисья': 48,
 'аонид': 49,
 'аполлона': 50,
 'аппетит': 51,
 'аптеки': 52,
 'апулея': 53,
 'аристократов': 54,
 'армейский': 55,
 'армид': 56,
 'архивны': 57,
 'аршин': 58,
 'атаман': 59,
 'африки': 60,
 'ах': 61,
 'б': 62,
 'баб': 63,
 'бабушкам': 64

In [15]:
text_as_int

array([  22, 6389, 5786, ..., 4296, 3403, 2747])

In [16]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text.split())//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(5):
    print(idx2word[i.numpy()])

александр
сергеевич
пушкин
евгений
онегин


In [17]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(' '.join(idx2word[item.numpy()])))

'александр сергеевич пушкин евгений онегин роман в стихах не мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных небрежный плод моих забав бессонниц легких вдохновений незрелых и увядших лет ума холодных наблюдений и сердца горестных замет глава первая и жить торопится и чувствовать спешит кн вяземский мой дядя самых честных правил когда не в шутку занемог он уважать себя заставил и лучше выдумать не мог его пример'
'другим наука но боже мой какая скука с больным сидеть и день и ночь не отходя ни шагу прочь какое низкое коварство полуживого забавлять ему подушки поправлять печально подносить лекарство вздыхать и думать про себя когда же черт возьмет тебя так думал молодой повеса летя в пыли на почтовых всевышней волею зевеса наследник 

In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'александр сергеевич пушкин евгений онегин роман в стихах не мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных небрежный плод моих забав бессонниц легких вдохновений незрелых и увядших лет ума холодных наблюдений и сердца горестных замет глава первая и жить торопится и чувствовать спешит кн вяземский мой дядя самых честных правил когда не в шутку занемог он уважать себя заставил и лучше выдумать не мог его'
Target data: 'сергеевич пушкин евгений онегин роман в стихах не мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав

In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [21]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [23]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1070848   
_________________________________________________________________
gru (GRU)                    (None, None, 1024)        3545088   
_________________________________________________________________
gru_1 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_2 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
dense (Dense)                (None, None, 8366)        8575150   
Total params: 25,786,286
Trainable params: 25,786,286
Non-trainable params: 0
_________________________________________________________________


In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 8366) # (batch_size, sequence_length, vocab_size)


In [25]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 8366), dtype=float32, numpy=
array([[-1.8984637e-04, -3.3298892e-04,  2.5663618e-04, ...,
        -6.2493644e-05, -6.4524422e-05,  8.5872496e-05],
       [-3.3925007e-05, -1.8819270e-04,  5.5039040e-04, ...,
         5.3059550e-05,  1.4051564e-04,  1.7408125e-05],
       [-3.8950070e-06, -5.4103086e-05,  4.6021130e-04, ...,
        -2.3460879e-04,  1.5867880e-04, -2.6035801e-05],
       ...,
       [-3.1206556e-04, -1.8072044e-04,  5.2405155e-04, ...,
        -2.5281275e-04,  8.4189884e-04, -2.4741053e-04],
       [-6.6577632e-05, -9.6959584e-06,  1.1120733e-03, ...,
        -4.2390812e-04,  8.0425100e-04, -1.5196278e-04],
       [-2.4737456e-04,  4.9004648e-05,  1.2008230e-03, ...,
        -6.1303983e-04,  7.2833523e-04, -3.3847644e-04]], dtype=float32)>

In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [27]:
print("Input: \n", repr(" ".join(idx2word[input_example_batch[0]])))
print()
print("Next Word Predictions: \n", repr(" ".join(idx2word[sampled_indices ])))

Input: 
 'с послом испанским говорит князь на онегина глядит ага давно ж ты не был в свете постой тебя представлю я да кто ж она жена моя так ты женат не знал я ране давно ли около двух лет на ком на лариной татьяне ты ей знаком я им сосед о так пойдем же князь подходит к своей жене и ей подводит родню и друга своего княгиня смотрит на него и что ей душу ни смутило как сильно ни была она удивлена поражена но ей ничто не изменило в ней сохранился тот же тон был так же тих ее поклон ейей'

Next Word Predictions: 
 'воспевал привел дружатся соблазнительных пел столе увял знай полям довольна октав зыбей скованный младое знаменитый сестра рассеянную старой младую брала светская упал громом твердят заменил ново заслужи достопочтенный покажешь завтрак карандаши пустой любопытная оживил поешь доле промолвил живые отдаю недруги стих безотрадно мешать праха изнывая соседок львы одно конечно плохой служанок дар нельзя холод мечтательной напев такова ручью задумчиво жадною впечатленье приникнул бы

In [28]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 8366)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.031919


In [29]:
model.compile(optimizer='adam', loss=loss)

In [30]:
# Directory where the checkpoints will be saved
# checkpoint_path = "./training_checkpoints/cp.ckpt_{epoch}"
checkpoint_path = "./training_checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)



checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True, verbose = 1)

In [31]:
EPOCHS = 500

# early_stopping=EarlyStopping(monitor='loss')
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/500
3/3 [==============================] - 4s 575ms/step - loss: 9.0261

Epoch 00001: saving model to ./training_checkpoints\cp.ckpt
Epoch 2/500
3/3 [==============================] - 2s 582ms/step - loss: 8.6948

Epoch 00002: saving model to ./training_checkpoints\cp.ckpt
Epoch 3/500
3/3 [==============================] - 2s 593ms/step - loss: 8.0730

Epoch 00003: saving model to ./training_checkpoints\cp.ckpt
Epoch 4/500
3/3 [==============================] - 2s 590ms/step - loss: 8.0112

Epoch 00004: saving model to ./training_checkpoints\cp.ckpt
Epoch 5/500
3/3 [==============================] - 2s 588ms/step - loss: 7.9618

Epoch 00005: saving model to ./training_checkpoints\cp.ckpt
Epoch 6/500
3/3 [==============================] - 2s 590ms/step - loss: 7.9209

Epoch 00006: saving model to ./training_checkpoints\cp.ckpt
Epoch 7/500
3/3 [==============================] - 2s 594ms/step - loss: 7.9180

Epoch 00007: saving model to ./training_checkpoints\cp.ckpt
Epoch 8/500
3

Epoch 59/500
3/3 [==============================] - 2s 592ms/step - loss: 6.9191

Epoch 00059: saving model to ./training_checkpoints\cp.ckpt
Epoch 60/500
3/3 [==============================] - 2s 592ms/step - loss: 6.9123

Epoch 00060: saving model to ./training_checkpoints\cp.ckpt
Epoch 61/500
3/3 [==============================] - 2s 594ms/step - loss: 6.9171

Epoch 00061: saving model to ./training_checkpoints\cp.ckpt
Epoch 62/500
3/3 [==============================] - 2s 593ms/step - loss: 6.8973

Epoch 00062: saving model to ./training_checkpoints\cp.ckpt
Epoch 63/500
3/3 [==============================] - 2s 593ms/step - loss: 6.7951

Epoch 00063: saving model to ./training_checkpoints\cp.ckpt
Epoch 64/500
3/3 [==============================] - 2s 593ms/step - loss: 6.7583

Epoch 00064: saving model to ./training_checkpoints\cp.ckpt
Epoch 65/500
3/3 [==============================] - 2s 594ms/step - loss: 6.6811

Epoch 00065: saving model to ./training_checkpoints\cp.ckpt
Epoch 

Epoch 117/500
3/3 [==============================] - 2s 594ms/step - loss: 4.0852

Epoch 00117: saving model to ./training_checkpoints\cp.ckpt
Epoch 118/500
3/3 [==============================] - 2s 590ms/step - loss: 4.0633

Epoch 00118: saving model to ./training_checkpoints\cp.ckpt
Epoch 119/500
3/3 [==============================] - 2s 591ms/step - loss: 4.0214

Epoch 00119: saving model to ./training_checkpoints\cp.ckpt
Epoch 120/500
3/3 [==============================] - 2s 593ms/step - loss: 3.9790

Epoch 00120: saving model to ./training_checkpoints\cp.ckpt
Epoch 121/500
3/3 [==============================] - 2s 594ms/step - loss: 3.9734

Epoch 00121: saving model to ./training_checkpoints\cp.ckpt
Epoch 122/500
3/3 [==============================] - 2s 595ms/step - loss: 3.9598

Epoch 00122: saving model to ./training_checkpoints\cp.ckpt
Epoch 123/500
3/3 [==============================] - 2s 594ms/step - loss: 3.8854

Epoch 00123: saving model to ./training_checkpoints\cp.ckpt

3/3 [==============================] - 2s 598ms/step - loss: 2.5877

Epoch 00174: saving model to ./training_checkpoints\cp.ckpt
Epoch 175/500
3/3 [==============================] - 2s 597ms/step - loss: 2.6019

Epoch 00175: saving model to ./training_checkpoints\cp.ckpt
Epoch 176/500
3/3 [==============================] - 2s 597ms/step - loss: 2.5452

Epoch 00176: saving model to ./training_checkpoints\cp.ckpt
Epoch 177/500
3/3 [==============================] - 2s 603ms/step - loss: 2.5270

Epoch 00177: saving model to ./training_checkpoints\cp.ckpt
Epoch 178/500
3/3 [==============================] - 2s 596ms/step - loss: 2.4562

Epoch 00178: saving model to ./training_checkpoints\cp.ckpt
Epoch 179/500
3/3 [==============================] - 2s 594ms/step - loss: 2.5081

Epoch 00179: saving model to ./training_checkpoints\cp.ckpt
Epoch 180/500
3/3 [==============================] - 2s 592ms/step - loss: 2.4564

Epoch 00180: saving model to ./training_checkpoints\cp.ckpt
Epoch 181/500

3/3 [==============================] - 2s 593ms/step - loss: 1.8222

Epoch 00231: saving model to ./training_checkpoints\cp.ckpt
Epoch 232/500
3/3 [==============================] - 2s 594ms/step - loss: 1.8067

Epoch 00232: saving model to ./training_checkpoints\cp.ckpt
Epoch 233/500
3/3 [==============================] - 2s 595ms/step - loss: 1.7685

Epoch 00233: saving model to ./training_checkpoints\cp.ckpt
Epoch 234/500
3/3 [==============================] - 2s 596ms/step - loss: 1.7014

Epoch 00234: saving model to ./training_checkpoints\cp.ckpt
Epoch 235/500
3/3 [==============================] - 2s 598ms/step - loss: 1.6992

Epoch 00235: saving model to ./training_checkpoints\cp.ckpt
Epoch 236/500
3/3 [==============================] - 2s 593ms/step - loss: 1.6591

Epoch 00236: saving model to ./training_checkpoints\cp.ckpt
Epoch 237/500
3/3 [==============================] - 2s 594ms/step - loss: 1.6470

Epoch 00237: saving model to ./training_checkpoints\cp.ckpt
Epoch 238/500

3/3 [==============================] - 2s 597ms/step - loss: 1.0240

Epoch 00288: saving model to ./training_checkpoints\cp.ckpt
Epoch 289/500
3/3 [==============================] - 2s 599ms/step - loss: 0.9902

Epoch 00289: saving model to ./training_checkpoints\cp.ckpt
Epoch 290/500
3/3 [==============================] - 2s 599ms/step - loss: 0.9887

Epoch 00290: saving model to ./training_checkpoints\cp.ckpt
Epoch 291/500
3/3 [==============================] - 2s 600ms/step - loss: 0.9747

Epoch 00291: saving model to ./training_checkpoints\cp.ckpt
Epoch 292/500
3/3 [==============================] - 2s 595ms/step - loss: 0.9896

Epoch 00292: saving model to ./training_checkpoints\cp.ckpt
Epoch 293/500
3/3 [==============================] - 2s 599ms/step - loss: 0.9668

Epoch 00293: saving model to ./training_checkpoints\cp.ckpt
Epoch 294/500
3/3 [==============================] - 2s 601ms/step - loss: 0.9740

Epoch 00294: saving model to ./training_checkpoints\cp.ckpt
Epoch 295/500

3/3 [==============================] - 2s 596ms/step - loss: 0.6496

Epoch 00345: saving model to ./training_checkpoints\cp.ckpt
Epoch 346/500
3/3 [==============================] - 2s 599ms/step - loss: 0.6389

Epoch 00346: saving model to ./training_checkpoints\cp.ckpt
Epoch 347/500
3/3 [==============================] - 2s 598ms/step - loss: 0.6504

Epoch 00347: saving model to ./training_checkpoints\cp.ckpt
Epoch 348/500
3/3 [==============================] - 2s 596ms/step - loss: 0.6393

Epoch 00348: saving model to ./training_checkpoints\cp.ckpt
Epoch 349/500
3/3 [==============================] - 2s 596ms/step - loss: 0.6242

Epoch 00349: saving model to ./training_checkpoints\cp.ckpt
Epoch 350/500
3/3 [==============================] - 2s 598ms/step - loss: 0.6167

Epoch 00350: saving model to ./training_checkpoints\cp.ckpt
Epoch 351/500
3/3 [==============================] - 2s 597ms/step - loss: 0.6222

Epoch 00351: saving model to ./training_checkpoints\cp.ckpt
Epoch 352/500

3/3 [==============================] - 2s 593ms/step - loss: 0.4026

Epoch 00402: saving model to ./training_checkpoints\cp.ckpt
Epoch 403/500
3/3 [==============================] - 2s 596ms/step - loss: 0.4047

Epoch 00403: saving model to ./training_checkpoints\cp.ckpt
Epoch 404/500
3/3 [==============================] - 2s 593ms/step - loss: 0.3990

Epoch 00404: saving model to ./training_checkpoints\cp.ckpt
Epoch 405/500
3/3 [==============================] - 2s 598ms/step - loss: 0.4010

Epoch 00405: saving model to ./training_checkpoints\cp.ckpt
Epoch 406/500
3/3 [==============================] - 2s 598ms/step - loss: 0.3918

Epoch 00406: saving model to ./training_checkpoints\cp.ckpt
Epoch 407/500
3/3 [==============================] - 2s 596ms/step - loss: 0.3841

Epoch 00407: saving model to ./training_checkpoints\cp.ckpt
Epoch 408/500
3/3 [==============================] - 2s 596ms/step - loss: 0.3863

Epoch 00408: saving model to ./training_checkpoints\cp.ckpt
Epoch 409/500

3/3 [==============================] - 2s 598ms/step - loss: 0.2428

Epoch 00459: saving model to ./training_checkpoints\cp.ckpt
Epoch 460/500
3/3 [==============================] - 2s 592ms/step - loss: 0.2369

Epoch 00460: saving model to ./training_checkpoints\cp.ckpt
Epoch 461/500
3/3 [==============================] - 2s 595ms/step - loss: 0.2384

Epoch 00461: saving model to ./training_checkpoints\cp.ckpt
Epoch 462/500
3/3 [==============================] - 2s 595ms/step - loss: 0.2329

Epoch 00462: saving model to ./training_checkpoints\cp.ckpt
Epoch 463/500
3/3 [==============================] - 2s 595ms/step - loss: 0.2363

Epoch 00463: saving model to ./training_checkpoints\cp.ckpt
Epoch 464/500
3/3 [==============================] - 2s 594ms/step - loss: 0.2352

Epoch 00464: saving model to ./training_checkpoints\cp.ckpt
Epoch 465/500
3/3 [==============================] - 2s 597ms/step - loss: 0.2270

Epoch 00465: saving model to ./training_checkpoints\cp.ckpt
Epoch 466/500

In [32]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\cp.ckpt'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [34]:
def generate_text(model, start_string, tmprt):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx[s] for s in start_string.split()]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = tmprt

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [35]:
text_ = generate_text(model, start_string=u"враги стоят потупя взор ", tmprt = 1)
print(text_)

враги стоят потупя взор враги стоят спесь люблю другой усердие липовых герою тотчас жернов какая рана рассудка заводов снегом навсегда стороны я сегодня мук вдруг издалека выходит ушей дворовая великолепные дуэль спесь не дают верила снегом путь желанный эпиграммы он мой все другой сергеевич он от пистолета спесь вас онегин издалека творческие вдруг издалека творческие выходит ушей дворовая великолепные блюдечках незнакомый с берегом и заточенье неугомонно жернов послушная незнакомый в руке ушей второму хоть покажем жернов послушная незнакомый бурные нетерпеливо подушкой ушей выходит воскресил творческие вдруг получил верила снегом тотчас герою тотчас хоть блюдечках незнакомый в стихи забор ушей вдруг издалека творческие вдруг получил вдруг издалека творческие липовых тотчас жернов послушная незнакомый озарит мой и жизни или плод путь кушал ревнивых берегом и сохраненная значит хоть блюдечках незнакомый чтоб траурной под подушкой ушей дворовая великолепные дуэль спесь в вышнем слезой к

In [36]:
text_ = generate_text(model, start_string=u"враги стоят потупя взор ", tmprt = 0.1)
print(text_)

враги стоят потупя взор враги стоят спесь не радо ушей дворовая великолепные дуэль спесь не радо ушей дворовая великолепные блюдечках незнакомый озарит в вышнем слезой какая рана тень стоят спесь не дают форейтор стороны таким вдруг издалека ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль спесь не радо ушей дворовая великолепные дуэль спесь не радо ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль спесь не дают верила снегом навсегда эпиграммы стоят спесь не радо ушей дворовая великолепные дуэль спесь не радо ушей дворовая великолепные блюдечках незнакомый онегин добро наук воскресил творческие выходит ушей дворовая великолепные блюдечках незнакомый озарит в вышнем слезой какая рана тень стоят спесь не радо ушей дворовая великолепные блюдечках незнакомый озарит в вышнем слезой какая рана тень стоят спесь не радо ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные блюдечках незнако

In [37]:
text_ = generate_text(model, start_string=u"враги стоят потупя взор ", tmprt = 0.001)
print(text_)

враги стоят потупя взор враги стоят спесь не радо ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дворовая великолепные дуэль выходит ушей дв

In [38]:
text_ = generate_text(model, start_string=u"враги стоят потупя взор ", tmprt = 10)
print(text_)

враги стоят потупя взор сладкозвучных летучей вспыхнул стыдом дом умней н лицы успех уходят цевницы необходимые важный отказ подумать волка бледнеть чудаков открыты бородой вечный сраженье кончен бородатый граненом бремя ваня столом кровавы столом кровавы наука басен паркет варенье золою устах слегка преследуют готовый опор благодарю владимир рога журнал нравился письмам улиц мурлыча теснятся мадригалы вымыслов целы неосторожно висок ума мы свинец засмеется сохнут княжны чувствительные улететь повеленьем реке живет оскорбляет благочестия близ соврать кибитки поражает всякий стыда легко оплошного благодарит ремянный живой пренебрегал готовил зовет божий слагает изнывая завидим муза огней разрешат евгений пустынным шомпол обрадован альбомах слез предлагает гораций кочующий покинуть произвол деревенские прикрасой гусар светло глядел снов явилися ожесточиться бежит помышлений которой темному любовницы носил усеян найдет все средь кончину выше душеньки кругом прочел догадливый чаю несносный

In [39]:
text_ = generate_text(model, start_string=u"враги стоят потупя взор ", tmprt = 100)
print(text_)

враги стоят потупя взор страдальческая пугая домашней тюфяки модой видя обедывал разыгранный ее хозяйский остротою ближний знаменитый перебирая веках навстречу страстное купец поселянам приездом грудь волнуя свежий занять столе светского убивать молвила спору верного них цветникам рука заглядывал резкую слезами именах жаль узнал мужчин вприсядку преданья цветам немых денницы желудок пускается развлечен дубравы пеньем тесноту мадоне простотой добровольный закидаем размышлению осанкой благородное носили как раб открылись прилетит устах напрасно машет опасные била поведеньем ревнивых сумрачной лестью перестань вериги голубка угадать монаха увянуть мольбы самовару заметя чистом ничтожность любопытным герои бледно ошибкой быстро очки снеговой поскакали вырваться поклонников воскресить странен феокрита начинает знатока убогий плач взор куклы роскошное подробно возражений запылал зайчик странной кубок тупо могиле усладила умной навещать капать чудесный ночь разлюбил дуэль понемногу точил меха

С уменьшением температуры модель зацикливается на одних и тех же словах, с повышением - набор становится более разнообразным